##논문 구현한 GITHUB
[깃허브 링크](https://github.com/KellerJordan/Autoencoder-Clustering/blob/master/Auto-Encoder%20Based%20Data%20Clustering.ipynb)

##아래 코드에서 사용된 알고리즘 흐름
**목표 : Autoencoder 에서 enocder를 통과한 같은 label의 data는 latent space 내에서 서로 가까이에 존재하도록**

Pretraining
1. 1epoch 동안 신경망 학습
2. 학습된 신경망으로 부터 0-9 까지의 label 가지는 data를 encoding 시켜서 초기 중심을 생성
------
알고리즘 시작
1. 1epoch 동안 batch data로 MSE와 Cluster_loss 생성 -> 전체 loss를 바탕으로 신경망 매개변수들 update
2. 전체 training data(60000개) 에 대해 기존의 중심들에 대해 모두 할당
3. 할당된 data를 바탕으로 새로운 중심들 생성
------
결론

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
transform = transforms.ToTensor() 
# 이미지를 pytorch tensor 로 변환
mnist_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
print(len(mnist_data)) # 60000
# tensor들로 변환된 mnist_data를 64크기의 batch로 random하게 뽑기
# shuffle 은 매 epoch마다 data가 shuffle 되도록
data_loader = torch.utils.data.DataLoader(dataset=mnist_data, batch_size=64, shuffle=True) # 60000장의 data를 64 장씩 random 으로 뽑아서 총 937개가 있는 data -> data와 label 로 구성되어 있음

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

60000


In [3]:
dataiter = iter(data_loader)
images, labels = dataiter.next() # batch size가 64인 data [64,1,28,28]
print(images.shape) # torch.Size([64, 1, 28, 28])
print(labels.shape) 
# 1개 image의 pixel값이 어떤 범위에 있는지 ,# output 출력할 때 활성화 함수 정하기 위해 필요
print(torch.min(images), torch.max(images)) 

torch.Size([64, 1, 28, 28])
torch.Size([64])
tensor(0.) tensor(1.)


In [4]:
# nn.Sequential 짤 때, 콤마(,) 점(.) 으로 찍는 것 조심
class Autoencoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder=nn.Sequential(
        nn.Linear(784, 250),
        nn.ReLU(),
        nn.Linear(250,50),
        nn.ReLU(),
        nn.Linear(50,10)  # (N,10) 
    )

    self.decoder=nn.Sequential(
        nn.Linear(10,50),
        nn.ReLU(),
        nn.Linear(50,250),
        nn.ReLU(),
        nn.Linear(250,784),
        nn.Sigmoid()  # 입력이 0-1 사이 이므로 출력도 0-1 사이가 되도록
    )

  # 복원된 image 구하기
  def forward(self,x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return encoded, decoded # (encoded, decoded) 나란히 튜플로 나오게끔

  # embedding -> latent vector 구하기
  def encoding(self,x):
    encoded = self.encoder(x)
    return encoded

  # forwarding -> recon 구하기
  def forwarding(self,x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [5]:
# Autoencdoer 모델 생성
model=Autoencoder()

In [6]:
# 가장 가까운 key 값 return 하는 함수
# dictinary에 'C0' , 'C1' ... 이 KEY로 들어가고 value 가 tensor 로 만들고 정렬해서 최소값에 해당하는 key 뽑기
def closest_center(centroids, encoded_x):
  distances={}
  for key in centroids.keys():
    distance=torch.sum((centroids[key] - encoded_x)**2).item()
    distances[key]=distance
  # print(distances) # {'C0': 0.0, 'C1': 0.012900883331894875, 'C2': 0.006580686662346125 .... }
  return sorted(distances.items(), key=lambda x:x[1])[0][0]  # [('C3', 0.0001), ('CO', 0.01), ('C1', 0.2), ... ] 에서 첫 번째 튜플의 첫번 째 원소

# print(closest_center(init_centroids, model.encoding(mnist_data[1][0].view(-1,28*28)))) # 'str' type

In [7]:
# Batch 로 넘어온 data (64,10) 을 하나하나 중심들과의 거리를 계산하고 최솟값을 더해간다
def clustering_loss(centroids, encoded_x):
  temp_cluster_loss=0
  for i in range(encoded_x.shape[0]):
    distances=[]
    for key in centroids.keys():
      distance=torch.sum((centroids[key]-encoded_x[i])**2).item()
      distances.append(distance)
    temp_cluster_loss+=min(distances)
  return temp_cluster_loss

In [8]:
# 가장 가까운 key 값 return 하는 함수
# dictinary에 'C0' , 'C1' ... 이 KEY로 들어가고 value 가 tensor 로 만들고 정렬해서 최소값에 해당하는 key 뽑기
def closest_center(centroids, encoded_x):
  distances={}
  for key in centroids.keys():
    distance=torch.sum((centroids[key] - encoded_x)**2).item()
    distances[key]=distance
  # print(distances) # {'C0': 0.0, 'C1': 0.012900883331894875, 'C2': 0.006580686662346125 .... }
  return sorted(distances.items(), key=lambda x:x[1])[0][0]  # [('C3', 0.0001), ('CO', 0.01), ('C1', 0.2), ... ] 에서 첫 번째 튜플의 첫번 째 원소

In [9]:
# accuracy 뽑아내는 함수
# 일단은 각 중심에 속한 data 개수 뽑아내는 함수
def print_accuracy_and_distribution(assignments):
  cluster_accuracy={}
  cluster_distribution={}
  index=0
  for key in assignments.keys():
    cluster_accuracy[key]={}
    cluster_distribution[key]={}
    sum=0
    for i in assignments[key].keys():
      sum+=len(assignments[key][i])
      cluster_distribution[key][i]=len(assignments[key][i])
    cluster_accuracy[key]= len(assignments[key][index])/sum
    index+=1
  
  print(cluster_accuracy)
  print("\n")
  print(cluster_distribution)
  print("\n")
 

In [10]:
# 1 epoch 만 먼저 돌려 놓기
def pretrain(criterion,optimizer):
  for img,label in data_loader:
    img=img.view(-1,28*28)
    recon=model.forwarding(img)
    loss=criterion(img,recon) 

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [11]:
criterion = nn.MSELoss() # Mean Squared Error 가 Loss function이 되도록 함 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3) # parameter를 갱신하는 것을 optimizer라고 함
# 미리 1 epoch 돌려 놓기
pretrain(criterion, optimizer)

In [12]:
# heuristic 한 방식으로 label 값 찾기 -> mnist_data는 순서가 정해져 있음 
print(mnist_data[1][1]) # 0
print(mnist_data[3][1]) # 1
print(mnist_data[5][1]) # 2
print(mnist_data[7][1]) # 3
print(mnist_data[2][1]) # 4
print(mnist_data[0][1]) # 5
print(mnist_data[13][1]) # 6
print(mnist_data[15][1]) # 7
print(mnist_data[17][1]) # 8
print(mnist_data[4][1]) # 9

0
1
2
3
4
5
6
7
8
9


In [13]:
# heuristic 한 방식으로 초기 중심점 잡기
c_0 = model.encoding(mnist_data[1][0].view(-1,28*28)) #0
c_1 = model.encoding(mnist_data[3][0].view(-1,28*28)) #1
c_2 = model.encoding(mnist_data[5][0].view(-1,28*28)) #2
c_3 = model.encoding(mnist_data[7][0].view(-1,28*28)) #3
c_4 = model.encoding(mnist_data[2][0].view(-1,28*28)) #4
c_5 = model.encoding(mnist_data[0][0].view(-1,28*28)) #5
c_6 = model.encoding(mnist_data[13][0].view(-1,28*28)) #6
c_7 = model.encoding(mnist_data[15][0].view(-1,28*28)) #7
c_8 = model.encoding(mnist_data[17][0].view(-1,28*28)) #8
c_9 = model.encoding(mnist_data[4][0].view(-1,28*28)) #9

print(c_0)

tensor([[-3.9707, -2.8988, -4.6210,  4.3977,  1.9660,  4.1361,  4.1452,  4.4857,
          3.4791, -3.8913]], grad_fn=<AddmmBackward0>)


In [14]:
torch.randn(10)

tensor([-2.5967, -1.4196, -1.0088,  0.4821,  0.7850,  0.3002, -0.0466, -0.2129,
        -0.9780, -2.3821])

In [15]:
# 초기 중심값 선정방법 1 -> 0-9 까지의 label data 를 encoding 시킨 것 
# 초기 중심값 선정방법 2 -> torch.rand() : 0과 1 사이의 숫자를 균등하게 생성 or torch.randn() : 평균이 0이고 표준편차가 1인 가우시안 정규분포를 이용해 생성
init_centroids={}
for i in range(10):
  init_centroids['C{}'.format(i)]=0 # 딕셔너리 key 값 동적생성 
init_centroids['C0']=c_0
init_centroids['C1']=c_1
init_centroids['C2']=c_2
init_centroids['C3']=c_3
init_centroids['C4']=c_4
init_centroids['C5']=c_5
init_centroids['C6']=c_6
init_centroids['C7']=c_7
init_centroids['C8']=c_8
init_centroids['C9']=c_9
print(init_centroids)


{'C0': tensor([[-3.9707, -2.8988, -4.6210,  4.3977,  1.9660,  4.1361,  4.1452,  4.4857,
          3.4791, -3.8913]], grad_fn=<AddmmBackward0>), 'C1': tensor([[-5.9411, -4.3384, -6.9145,  6.5770,  2.9420,  6.1983,  6.2133,  6.7294,
          5.2066, -5.8322]], grad_fn=<AddmmBackward0>), 'C2': tensor([[-5.9654, -4.3562, -6.9427,  6.6038,  2.9540,  6.2237,  6.2388,  6.7570,
          5.2279, -5.8561]], grad_fn=<AddmmBackward0>), 'C3': tensor([[-6.9885, -5.1037, -8.1337,  7.7355,  3.4609,  7.2946,  7.3127,  7.9221,
          6.1250, -6.8640]], grad_fn=<AddmmBackward0>), 'C4': tensor([[-2.6765, -1.9533, -3.1144,  2.9662,  1.3249,  2.7815,  2.7868,  3.0120,
          2.3443, -2.6164]], grad_fn=<AddmmBackward0>), 'C5': tensor([[-6.0005, -4.3818, -6.9837,  6.6427,  2.9715,  6.2605,  6.2757,  6.7971,
          5.2587, -5.8907]], grad_fn=<AddmmBackward0>), 'C6': tensor([[-6.4213, -4.6893, -7.4734,  7.1081,  3.1799,  6.7009,  6.7173,  7.2762,
          5.6276, -6.3052]], grad_fn=<AddmmBackward0>)

In [119]:
# Training 과정
# 1 epoch 동안 batch 단위로 w,b값이 update 되면서 동시에 cluster loss 또한 update 되어감
def training(criterion, optimizer, T, cluster_centroids):
  for t in range(T):
    for img,label in data_loader: # 1epoch 끝나고 나서 shuffle 되고 1epoch 내에서는 차례로 다 나옴 (중복 x)
      cluster_loss=0
      img=img.view(-1,784) # torch.Size([64, 784])
      encoded_data, recon =model.forward(img)
      MSE = criterion(img,recon)
      cluster_loss += clustering_loss(cluster_centroids,encoded_data)
      loss = MSE + 0.1*cluster_loss # 일단 MSE + cluster_loss

      optimizer.zero_grad() # 배치마다 gradient 를 0으로 초기화
      loss.backward()
      optimizer.step()

    # 이전 중심에 할당하는 작업
    # 10 은 label 개수
    # 할당되는 값은 index를 넣어서 좀 더 효율적으로 계산할 수 있게끔 한다
    cluster_assignments={}
    for i in range(10):
      cluster_assignments['C{}'.format(i)]={} # 딕셔너리 key 값 동적생성 
    for key in cluster_assignments.keys():
      for i in range(10):
        cluster_assignments[key][i]=[]

    for index,(img,label) in enumerate(mnist_data):
      new_img=img.view(-1,784)
      encoded_data=model.encoding(new_img)
      key = closest_center(cluster_centroids, encoded_data)
      dictionary = cluster_assignments[key]
      dictionary[label].append(index)
    
    # 중심 update -> 60000 장에 대해서 중심
    new_cluster={}
    for i in range(10):
      new_cluster['C{}'.format(i)]={} # 딕셔너리 key 값 동적생성 
    
    for key in cluster_assignments.keys():
      sum = torch.tensor([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])

      
      count=0
      for i in cluster_assignments[key].keys():
        count+=len(cluster_assignments[key][i]) # 해당 label 의 개수
        for index in cluster_assignments[key][i]: # list
          sum+=model.encoding(mnist_data[index][0].view(-1,784)) # 해당 index의 image를 encoding 시킨 값을 계속 더함
      
      new_cluster[key]=sum/count
    
    cluster_centroids=new_cluster # 기존의 중심을 새로운 중심으로 update

    # 1epoch 끝난 후 MSE loss 와 cluster loss 
    print(f'Iteration:{t+1}, Loss:{MSE:.4f}, cluster_loss:{cluster_loss}\n')
    print(new_cluster)
    print("\n")

    print_accuracy_and_distribution(cluster_assignments)

In [120]:
criterion = nn.MSELoss() # Mean Squared Error 가 Loss function이 되도록 함 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3) # parameter를 갱신하는 것을 optimizer라고 함
training(criterion,optimizer, 10, init_centroids)

Iteration:1, Loss:0.0702, cluster_loss:9335.06576538086

{'C0': tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]), 'C1': tensor([[ 5.3092, -4.9937,  6.0639,  8.0325,  5.2674, -5.0723, -0.8426,  5.4181,
          5.9407,  5.2209]], grad_fn=<DivBackward0>), 'C2': tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]), 'C3': tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]), 'C4': tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]), 'C5': tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]), 'C6': tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]), 'C7': tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]), 'C8': tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]]), 'C9': tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])}




ZeroDivisionError: ignored

In [ ]:
import numpy as np
batch_mask=np.random.choice(100,10)
batch_mask